<a href="https://colab.research.google.com/github/Vetrivel-PS/JanataHack-Computer-Vision-Hackathon/blob/master/CV_Hack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step 1 : Setting up Colab Notebook :

In [195]:
!pip install PyDrive

In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
download = drive.CreateFile({'id': '1VN_c2R2x-Il-YLDec6tmWOTJCFj2mKPB'})

In [0]:
# download.GetContentFile('train_SOaYf6m.zip')

In [0]:
# !unzip train_SOaYf6m.zip

### Step 2 : Import the libraries we’ll need during our model building phase 

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.optimizers import SGD
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

### Step 3: Recall the pre-processing steps we discussed earlier. We’ll be using them here after loading the data.

In [0]:
download.GetContentFile('train.csv')

In [0]:
train = pd.read_csv('train.csv')

In [204]:
download = drive.CreateFile({'id': '1tIb3n_N2YebNlTmzmGMnUawy5HdTLKo_'})
download.GetContentFile('train_SOaYf6m.zip')
!unzip train_SOaYf6m.zip

Archive:  train_SOaYf6m.zip
replace images/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: images/0.jpg            
  inflating: images/1.jpg            
  inflating: images/10.jpg           
  inflating: images/100.jpg          
  inflating: images/1000.jpg         
  inflating: images/1001.jpg         
  inflating: images/1002.jpg         
  inflating: images/1003.jpg         
  inflating: images/1004.jpg         
  inflating: images/1005.jpg         
  inflating: images/1006.jpg         
  inflating: images/1007.jpg         
  inflating: images/1008.jpg         
  inflating: images/1009.jpg         
  inflating: images/101.jpg          
  inflating: images/1010.jpg         
  inflating: images/1011.jpg         
  inflating: images/1012.jpg         
  inflating: images/1013.jpg         
  inflating: images/1014.jpg         
  inflating: images/1015.jpg         
  inflating: images/1016.jpg         
  inflating: images/1017.jpg         
  inflating: images/1018.jpg        

In [205]:
pwd

'/content'

In [206]:
ls

adc.json                   sample_cnn_2.csv  sample_cnn.csv
images/                    sample_cnn_3.csv  sample_data/
sample_cnn_10.csv          sample_cnn_4.csv  sample_submission_yxjOnvz.csv
sample_cnn_11.csv          sample_cnn_5.csv  test.csv
sample_cnn_12.csv          sample_cnn_6.csv  train.csv
sample_cnn_13.csv          sample_cnn_7.csv  train_SOaYf6m.zip
sample_cnn_14_Nadam.csv    sample_cnn_8.csv
sample_cnn_15_RMSProp.csv  sample_cnn_9.csv


In [207]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img('images/'+train['image_names'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)
# As it is a multi-class classification problem (10 classes), we will one-hot encode the target variable.

y=train['emergency_or_not'].values
y = to_categorical(y)

  0%|          | 0/1646 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 1646/1646 [00:02<00:00, 805.88it/s]


### Step 4: Creating a validation set from the training data.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 294)

### Step 5: Define the model structure.

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_uniform',input_shape=(28,28,1)))
model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [0]:
# opt = SGD(lr=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy',optimizer="sgd",metrics=['accuracy'])

### Step 6: Training the model.

In [293]:
model.fit(X_train, y_train, epochs=500,  validation_data=(X_test, y_test))

Train on 1399 samples, validate on 247 samples
Epoch 1/500
1399/1399 [==============================] - 0s 342us/step - loss: 0.8144 - accuracy: 0.5618 - val_loss: 0.6580 - val_accuracy: 0.5992
Epoch 2/500
1399/1399 [==============================] - 0s 186us/step - loss: 0.6801 - accuracy: 0.5733 - val_loss: 0.6561 - val_accuracy: 0.6194
Epoch 3/500
1399/1399 [==============================] - 0s 195us/step - loss: 0.6668 - accuracy: 0.5897 - val_loss: 0.6387 - val_accuracy: 0.6154
Epoch 4/500
1399/1399 [==============================] - 0s 189us/step - loss: 0.6523 - accuracy: 0.6040 - val_loss: 0.6230 - val_accuracy: 0.6964
Epoch 5/500
1399/1399 [==============================] - 0s 185us/step - loss: 0.6337 - accuracy: 0.6340 - val_loss: 0.6065 - val_accuracy: 0.6275
Epoch 6/500
1399/1399 [==============================] - 0s 188us/step - loss: 0.6216 - accuracy: 0.6412 - val_loss: 0.5831 - val_accuracy: 0.7449
Epoch 7/500
1399/1399 [==============================] - 0s 194us/step 

### Step 7: Making predictions!

In [0]:
download = drive.CreateFile({'id': '18eiyECVFkSy3w196yiMilEkgy0Or8o0_'})
download.GetContentFile('test.csv')

test = pd.read_csv('test.csv')

In [295]:
# We have grayscale images, so while loading the images we will keep grayscale=True, if you have RGB images, you should set grayscale as False
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('images/'+test['image_names'][i], target_size=(28,28,1), grayscale=True)
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)

  0%|          | 0/706 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|██████████| 706/706 [00:00<00:00, 821.67it/s]


In [0]:
# making predictions
prediction = model.predict_classes(test)

In [297]:
print(len(prediction))

706


### Step 8 : Creating Submission File :

In [0]:
download = drive.CreateFile({'id': '1n4DDHv6Jy5zKxJnUBCi4Jzi4ijH9kyaR'})
download.GetContentFile('sample_submission_yxjOnvz.csv')

In [0]:
file_name = "sample_cnn_22_500_Epochs_SGD_Stratify_NO_Dropout_0.20"

In [0]:
# creating submission file

sample = pd.read_csv('sample_submission_yxjOnvz.csv')
sample['emergency_or_not'] = prediction
sample.to_csv(file_name+'.csv', header=True, index=False)

In [0]:
from google.colab import files

In [0]:
files.download(file_name+'.csv')